In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/abalone1/abalone.csv
/kaggle/input/playground-series-s4e4/sample_submission.csv
/kaggle/input/playground-series-s4e4/train.csv
/kaggle/input/playground-series-s4e4/test.csv


In [2]:
train_dataset_root = r"/kaggle/input/playground-series-s4e4/train.csv"
train_dataset_root2 = r"/kaggle/input/abalone1/abalone.csv"
test_dataset_root = r"/kaggle/input/playground-series-s4e4/test.csv"
train_dataset = pd.read_csv(train_dataset_root)
# one hot
train_dataset = pd.get_dummies(train_dataset, columns=['Sex'])
test_dataset = pd.read_csv(test_dataset_root)
test_dataset = pd.get_dummies(test_dataset, columns=['Sex'])
# print(train_dataset.head(3))
train_dataset2 = pd.read_csv(train_dataset_root2)
train_dataset2 = pd.get_dummies(train_dataset2, columns=['Sex'])
train_dataset2 = train_dataset2.rename(columns={'Shucked weight': 'Whole weight.1', 'Viscera weight': 'Whole weight.2'})
print(train_dataset2.head(3))

   Length  Diameter  Height  Whole weight  Whole weight.1  Whole weight.2  \
0   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   0.530     0.420   0.135        0.6770          0.2565          0.1415   

   Shell weight  Rings  Sex_F  Sex_I  Sex_M  
0          0.15     15  False  False   True  
1          0.07      7  False  False   True  
2          0.21      9   True  False  False  


In [3]:
X = train_dataset.drop(columns=["id","Rings"]).values  # feature
X2 = train_dataset2.drop(columns=["Rings"]).values
print("Shape of X:", X.shape)
print("Shape of X2:", X2.shape)
X = np.concatenate((X, X2), axis=0)
y = train_dataset['Rings'].values  # label
y2 = train_dataset2['Rings'].values  # label
y = np.concatenate((y, y2), axis=0)
print(X)
print(X2)
print(y)

Shape of X: (90615, 10)
Shape of X2: (4177, 10)
[[0.55 0.43 0.15 ... True False False]
 [0.63 0.49 0.145 ... True False False]
 [0.16 0.11 0.025 ... False True False]
 ...
 [0.6 0.475 0.205 ... False False True]
 [0.625 0.485 0.15 ... True False False]
 [0.71 0.555 0.195 ... False False True]]
[[0.455 0.365 0.095 ... False False True]
 [0.35 0.265 0.09 ... False False True]
 [0.53 0.42 0.135 ... True False False]
 ...
 [0.6 0.475 0.205 ... False False True]
 [0.625 0.485 0.15 ... True False False]
 [0.71 0.555 0.195 ... False False True]]
[11 11  6 ...  9 10 12]


# **Preprocess**

In [4]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
class DNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

    
input_size = X_train.shape[1]
hidden_size1 = 64
hidden_size2 = 32
model = DNN(input_size, hidden_size1, hidden_size2)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 1000
for epoch in range(num_epochs):
    # 前向传播
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.view(-1, 1))

    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 每10个epoch打印一次训练信息
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
    
with torch.no_grad():
    val_outputs = model(X_val_tensor)
    val_loss = criterion(val_outputs, y_val_tensor.view(-1, 1))
    print(f'Validation Loss: {val_loss.item():.4f}')


Epoch 0, Loss: 105.9239
Epoch 10, Loss: 99.6708
Epoch 20, Loss: 91.4872
Epoch 30, Loss: 80.0641
Epoch 40, Loss: 64.9095
Epoch 50, Loss: 47.8558
Epoch 60, Loss: 33.4875
Epoch 70, Loss: 25.7193
Epoch 80, Loss: 22.9735
Epoch 90, Loss: 20.1944
Epoch 100, Loss: 17.2883
Epoch 110, Loss: 14.7081
Epoch 120, Loss: 12.3908
Epoch 130, Loss: 10.4628
Epoch 140, Loss: 8.9373
Epoch 150, Loss: 7.7768
Epoch 160, Loss: 6.8980
Epoch 170, Loss: 6.2612
Epoch 180, Loss: 5.8055
Epoch 190, Loss: 5.4815
Epoch 200, Loss: 5.2500
Epoch 210, Loss: 5.0807
Epoch 220, Loss: 4.9519
Epoch 230, Loss: 4.8498
Epoch 240, Loss: 4.7653
Epoch 250, Loss: 4.6928
Epoch 260, Loss: 4.6290
Epoch 270, Loss: 4.5720
Epoch 280, Loss: 4.5205
Epoch 290, Loss: 4.4739
Epoch 300, Loss: 4.4315
Epoch 310, Loss: 4.3927
Epoch 320, Loss: 4.3574
Epoch 330, Loss: 4.3250
Epoch 340, Loss: 4.2953
Epoch 350, Loss: 4.2683
Epoch 360, Loss: 4.2433
Epoch 370, Loss: 4.2201
Epoch 380, Loss: 4.1984
Epoch 390, Loss: 4.1780
Epoch 400, Loss: 4.1590
Epoch 410, L

In [5]:
X = test_dataset.drop(columns=["id"]).values
test_id = test_dataset["id"].values
print(X)
test_scaled = scaler.fit_transform(X)
test_tensor = torch.tensor(test_scaled, dtype=torch.float32)
with torch.no_grad():
    test_outputs = model(test_tensor).numpy()
result_df = pd.DataFrame({'id': test_id, 'Rings': test_outputs.flatten()})
# 将DataFrame保存到CSV文件中
result_df.to_csv('submission.csv', index=False)

[[0.645 0.475 0.155 ... False False True]
 [0.58 0.46 0.16 ... False False True]
 [0.56 0.42 0.14 ... False False True]
 ...
 [0.59 0.44 0.155 ... False True False]
 [0.66 0.525 0.19 ... True False False]
 [0.43 0.34 0.12 ... True False False]]
